In [ ]:
# Standard libraries of Python
import re

# Dependencies
import pandas as pd
import requests
from bs4 import BeautifulSoup, Tag
# from dataclasses import dataclass
from selectolax.parser import HTMLParser

# Libraries
from backend.database.parsers import parse_movements
from backend.database.src.creature import Pokemon, Mega_Pokemon
from backend.database.utils import functions

In [ ]:
# Class for selectolax scrap
# @dataclass
# class Pokemon:
#     name: str
#     gender: str
#     types: list

In [ ]:
# Selecto scrap
pokedex = []
url = 'https://www.serebii.net/pokedex-sm/{}.shtml'.format(str(1).zfill(3))
html = requests.get(url).text
tree = HTMLParser(html)
pokemon = Pokemon(name=tree.css_first('#content > main > div > div > table:nth-child(7) > tbody > tr:nth-child(2) > td:nth-child(1)').text(),
            gender=tree.css_first('#content > main > div > div > table:nth-child(7) > tbody > tr:nth-child(2) > td:nth-child(4)').text(),
            types=list(map(lambda x: x.attributes['alt'], tree.css('#content > main > div > div > table:nth-child(5) > tbody > tr:nth-child(2) > td.cen >> img'))))
pokedex.append(pokemon)

pd.DataFrame(pokedex)

In [ ]:
# Examble of use with bs4
html_text = requests.get(f'https://www.serebii.net/pokedex-xy/006.shtml').text
soup = BeautifulSoup(html_text, 'html.parser')
all_divs = soup.find_all('div', attrs={'align': 'center'})
foo_info = all_divs[0].find_all('td', {'class': 'footype'})

In [ ]:
# Class test
x = Pokemon(8,'charizard')

all_divs = x.soup.find_all('div', attrs={'align': 'center'})
foo_info = all_divs[1].find_all('table', {'class': 'dextable'})

x.name()
x.elements()

try:
    m = Mega_Pokemon(x)
except ValueError as e:
    print(f'{e}')

In [ ]:
scrap = parse_movements.list_composition(html=foo_info[12])

In [ ]:
ala = BeautifulSoup(str(scrap[0:10][7]), 'html.parser')

In [ ]:
if '-a' in scrap[10:22][9].get('src'):
    print('Alolan_form')

In [ ]:
parse_movements.attack_form_process(line=scrap[0:10],location_index=7,category='Pre evolution')

In [ ]:
ele = ala.find('img').get('src')

In [ ]:
scrap

In [ ]:
# Special moves that pokémon has through events
scrap = parse_movements.list_composition(html=foo_info[15])
regional = functions.normal_regional(x.p_elements)
df = parse_movements.make_it_table(scrap=scrap,category='Special Move',regional_form=regional)

In [ ]:
pd.DataFrame(df)

In [ ]:
# Transfer moves for regional and normal form
scrap = parse_movements.list_composition(html=foo_info[20])
regional = functions.normal_regional(x.p_elements)
moves, rmoves = parse_movements.transfer_moves(scrap,'Transfer Move',regional)

In [ ]:
pd.DataFrame(moves)

In [ ]:
pd.DataFrame(rmoves)

In [ ]:
# Move Tutor attacks scrap
scrap = parse_movements.list_composition(html=foo_info[13])
regional = functions.normal_regional(x.p_elements)
df = parse_movements.make_it_table(scrap=scrap,category='Move Tutor',regional_form=regional,pokemon_name=x.p_name)

In [ ]:
pd.DataFrame(df)

In [ ]:
# Egg moves in case of regional form pokémon, for the 8th generation and 7th generation, including BDSP data
scrap = parse_movements.list_composition(html=foo_info[12],category='Egg Move')
regional = functions.normal_regional(x.p_elements)
df = parse_movements.make_it_table(scrap=scrap,category='Egg Move',regional_form=regional)

In [ ]:
pd.DataFrame(df)

In [ ]:
# Pre-Evolution moves when applicable
scrap = parse_movements.list_composition(html=foo_info[18])
df = parse_movements.make_it_table(scrap=scrap,category='Pre_evolution')

In [ ]:
pd.DataFrame(df)

In [ ]:
# TM | TR | Technical Machine | Technical Record case for regional form pokémon in the 8th generation and 7th generation
scrap = parse_movements.list_composition(html=foo_info[12])
regional = functions.normal_regional(x.p_elements)
df = parse_movements.make_it_table(scrap=scrap,category='TM',regional_form=regional)

In [ ]:
pd.DataFrame(df)

In [ ]:
# Max Moves and Z Moves data scrap from Serebii.net, for pokemon with regional forms
scrap = parse_movements.list_composition(html=foo_info[17])
positions, group = parse_movements.obtain_positions(scrap)
main_table = parse_movements.define_table(group,positions,scrap)
regional = functions.normal_regional(x.p_elements)
df = parse_movements.make_it_table(scrap=main_table,category='Max Move',regional_form=regional)

In [ ]:
pd.DataFrame(df)

In [ ]:
#Weakness
types = []

for td_tag in foo_info[0:18]:
    a_tag = td_tag.find('img')
    if a_tag:
        type_text = a_tag['alt']
        types.append(type_text)

s = 'Attacking Move Type: ','-type'
for string in s:
    types = list(map(lambda x: x.replace(string,''),types))

print(types)

filtered = [tag for tag in foo_info[18:36] if '*' in tag.get_text(strip=True)]
filtered_t = [i.text for i in filtered]
val = list(map(lambda x: int(x.replace('*','')),filtered_t))

In [ ]:
#Elemental Types
tp = foo_info[0]
types = []
for tag in tp:
    a_tag = tag.find('img')
    print(a_tag)
    type_text = a_tag['alt']
    print(type_text)
    types.append(type_text)

s = 'Attacking Move Type: ','-type'
for string in s:
    types = list(map(lambda x: x.replace(string,''),types))

In [ ]:
base_form = []
form = []

if foo_info[0].find(string='Normal'):
    main = foo_info[0].find_all('td')

    for index,tag in enumerate(main):
        lazy = tag.text.strip()

        if lazy == 'Normal':
            basic = main[index+1].find('img')
            type_text = basic['alt']
            base_form.append(type_text)
        elif lazy:
            basic = main[index+1].find_all('img')
            for e in basic:
                type_text = e['alt']
                form.append(type_text)

In [ ]:
#Get al tables from the main center table
tables = all_divs[0].find_all('table', {'class': 'dextable'})

#Search for the line that contains <td class="fooevo" colspan="6">
mega_evolution_position = -1
counter = 0
for i, table in enumerate(tables):
    if table.find('td', {'class': 'fooevo', 'colspan': '6'}):
        mega_evolution_position = i
        print(i)
        print(table)
        break #Simulates return
    else:
        print('Does not have Mega')

In [ ]:
#Mega Evolution Types
t = tables[19].find('td', class_='cen')

ty_val = []

for i in t:
    n = i.find('img')
    if n and not isinstance(n,Tag):
        continue
    n = n['src']
    for element in x.elemental_types:
        ty = element.lower()
        if ty in n:
            ty_val.append(element)

print(ty_val)

In [ ]:
#Mega Evolution Weakness
w = []

if ty_val == x._elements:
    pass
else:
    weak_location = tables[22].find('td', string=re.compile("Base Stats - Total.*")).find_next_siblings('td')
    filtered_types = list(filter(lambda x: '*' in x.text, weak_location.find_all('td', {'class': 'footype'})))
    filtered = [tag.text for tag in filtered_types if '*' in tag.get_text(strip=True)]

    val = list(map(lambda x: x.replace('*',''),filtered_t))

    for i in val:
        try:
            w.append(int(i))
        except ValueError:
            w.append(float(i))

    mega_weak = dict(zip(x.elemental_types,w))